## Quickstart and check llm connectivity

In [1]:
from openai import OpenAI
import os
import json
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [2]:
client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": "Write a one-sentence bedtime story about a unicorn."
        }
    ]
)

print(completion.choices[0].message.content)

Once upon a time, a beautiful unicorn danced under the twinkling stars before curling up in a meadow of soft grass to dream of magical adventures.


## Actual function calling experiments

In [3]:
# Define the tools for function calling

# This function simulates fetching weather data and returns a JSON string.
def get_current_weather(location: str, unit: str ="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [4]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get current weather in a given location. Default is Celsius.",
            "strict": True,
            "parameters": {
                "type": "object",
                "required": [
                "location",
                "unit"
                ],
                "properties": {
                "location": {
                    "type": "string",
                    "description": "A string representing the geographic location for which to retrieve the weather."
                },
                "unit": {
                    "type": "string",
                    "description": "Temperature unit; defaults to 'Celsius'. Use 'Fahrenheit' for Fahrenheit.",
                    "enum": [
                    "Celsius",
                    "Fahrenheit"
                    ]
                }
                },
                "additionalProperties": False
            }
        }
    }
]

messages = [{"role": "user", "content": "What's the weather like in Paris today?"}]

completion_1 = client.chat.completions.create(
    model="gpt-4.1",
    messages=messages,
    tools=tools,
)

print(completion_1.choices[0].message.tool_calls)

[ChatCompletionMessageToolCall(id='call_AiTwxmCDh5STwPnEV3ToYgrM', function=Function(arguments='{"location":"Paris","unit":"Celsius"}', name='get_current_weather'), type='function')]


In [22]:
#  call the function based on the tool call
tool_call = completion_1.choices[0].message.tool_calls[0]
function_name = tool_call.function.name
args = json.loads(tool_call.function.arguments)

In [20]:
if function_name == "get_current_weather":
    result = get_current_weather(**args)
    print(result)


{"location": "Paris", "temperature": "72", "unit": "Celsius", "forecast": ["sunny", "windy"]}


In [21]:
messages.append(completion_1.choices[0].message)  # append model's function call message
messages.append({                               # append result message
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": str(result)
})

completion_2 = client.chat.completions.create(
    model="gpt-4.1",
    messages=messages,
    tools=tools,
)

completion_2.choices[0].message.content

'The weather in Paris today is sunny and windy, with a temperature of 72°C. If you want details in Fahrenheit or a more detailed forecast, let me know!'